In [4]:
class BaseModelAdapter:
    """
    Base class for language model adapters.
    
    This abstract class defines the interface that all model adapters
    should implement.
    """
    
    def __init__(self, model_type: str = "BaseModel"):
        """
        Initialize the model adapter.
        
        Args:
            model_type: Identifier for the model type
        """
        self._type = model_type
        
    @property
    def model_type(self) -> str:
        """Get the type identifier for this model adapter."""
        return self._type
        
    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """
        Make a synchronous call to the language model.
        
        Args:
            prompt: The prompt/question to send
            stop: Optional stop sequences
            **kwargs: Additional parameters
            
        Returns:
            Model's response as a string
        """
        raise NotImplementedError("Subclasses must implement _call method")
        
    async def _acall(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """
        Make an asynchronous call to the language model.
        
        Args:
            prompt: The prompt/question to send
            stop: Optional stop sequences
            **kwargs: Additional parameters
            
        Returns:
            Model's response as a string
        """
        raise NotImplementedError("Subclasses must implement _acall method")
        
    def invoke(self, input: str, config: Optional[Dict[str, Any]] = None) -> str:
        """
        Invoke the model with the given input.
        
        Args:
            input: The prompt to send to the model
            config: Optional configuration dictionary
            
        Returns:
            The model's response
        """
        # Extract parameters from config
        config = config or {}
        
        # Make the call with config parameters
        kwargs = {k: v for k, v in config.items()}
        return self._call(prompt=input, **kwargs)

    async def ainvoke(self, input: str, config: Optional[Dict[str, Any]] = None) -> str:
        """
        Asynchronously invoke the model with the given input.
        
        Args:
            input: The prompt to send to the model
            config: Optional configuration dictionary
            
        Returns:
            The model's response
        """
        # Extract parameters from config
        config = config or {}
        
        # Make the call with config parameters
        kwargs = {k: v for k, v in config.items()}
        return await self._acall(prompt=input, **kwargs)
        
    def batch(
        self, 
        inputs: List[str], 
        config: Optional[Dict[str, Any]] = None
    ) -> List[str]:
        """
        Process a batch of inputs.
        
        Args:
            inputs: List of prompts to process
            config: Optional configuration dictionary
            
        Returns:
            List of responses
        """
        return [self.invoke(input, config) for input in inputs]

    async def abatch(
        self, 
        inputs: List[str], 
        config: Optional[Dict[str, Any]] = None
    ) -> List[str]:
        """
        Asynchronously process a batch of inputs.
        
        Args:
            inputs: List of prompts to process
            config: Optional configuration dictionary
            
        Returns:
            List of responses
        """
        return [await self.ainvoke(input, config) for input in inputs]


class LLM:
    """
    LLM (Language Model) adapter for AWM infrastructure.
    
    This class provides an interface for interacting with language models
    through the CMS API.
    """
    
    def __init__(self, cms: Optional[CMS] = None):
        """
        Initialize the LLM adapter.
        
        Args:
            cms: Optional pre-configured CMS client instance
        """
        self.cms = cms
        self._type = "CMSLanguageModel"
        
    @property
    def model_type(self) -> str:
        """Get the type identifier for this model."""
        return self._type

    @classmethod
    def build_cms(
        cls,
        app_id: Optional[str] = None,
        env: Optional[str] = None,
        model_name: Optional[str] = None,
        temperature: Optional[float] = None,
        log_level: str = "INFO",
        **kwargs: Any,
    ) -> CMS:
        """
        Build a CMS client with the specified configuration.
        
        This factory method creates a properly configured CMS client with the
        given parameters and preference settings.
        
        Args:
            app_id: Application identifier
            env: Environment name (dev, uat, prod, etc.)
            model_name: Name of the inference model to use
            temperature: Temperature setting for generation
            log_level: Logging level for the CMS client
            **kwargs: Additional preference parameters
            
        Returns:
            Configured CMS client
        """
        # Get preferences with the specified model and temperature
        preferences = get_preferences(
            inference_model_name=model_name, 
            temperature=temperature, 
            **kwargs
        )

        # Create and return CMS client
        return CMS(
            app_id=app_id,
            env=env,
            preferences=preferences,
            log_level=log_level
        )

    @classmethod
    def init(
        cls,
        app_id: str,
        env: Optional[str] = None,
        model_name: Optional[str] = None,
        temperature: Optional[float] = None,
        log_level: str = "INFO",
        **kwargs: Any,
    ) -> "LLM":
        """
        Create a configured LLM adapter with a new CMS client.
        
        Factory method for creating an LLM adapter with sensible defaults.
        
        Args:
            app_id: Application identifier (required)
            env: Environment name
            model_name: Name of the inference model
            temperature: Temperature setting
            log_level: Logging level
            **kwargs: Additional preferences
            
        Returns:
            Configured LLM adapter
        """
        cms = cls.build_cms(
            app_id=app_id,
            env=env,
            model_name=model_name,
            temperature=temperature,
            log_level=log_level,
            **kwargs,
        )
        
        return cls(cms)
        
    # Stop tokens handling removed as it's unnecessary for API-based implementations
        
    def invoke(self, input: str, config: Optional[Dict[str, Any]] = None) -> str:
        """
        Invoke the model with the given input.
        
        Args:
            input: The prompt to send to the model
            config: Optional configuration dictionary
            
        Returns:
            The model's response
            
        Raises:
            ValueError: If CMS client is not initialized
        """
        if self.cms is None:
            raise ValueError("CMS client not initialized")
            
        # Extract parameters from config
        config = config or {}
        stop = config.get("stop")
        user_sso = config.get("user_sso")
        
        # Get authentication token if not provided
        if not user_sso:
            try:
                user_sso = gs_auth.get_sso()
                logger.debug("Using automatically retrieved sso token")
            except Exception as e:
                raise ValueError(f"Failed to get sso token: {str(e)}")
            
        # Call the CMS inference API
        answer, _, error = self.cms.inference(
            user_sso=user_sso, 
            question=input
        )

        # Handle errors
        if error is not None:
            raise RuntimeError(f"Error in inference: {error}")

        # Stop token handling is done server-side in the API
            
        return answer

    async def ainvoke(self, input: str, config: Optional[Dict[str, Any]] = None) -> str:
        """
        Asynchronously invoke the model with the given input.
        
        Args:
            input: The prompt to send to the model
            config: Optional configuration dictionary
            
        Returns:
            The model's response
            
        Raises:
            ValueError: If CMS client is not initialized
        """
        if self.cms is None:
            raise ValueError("CMS client not initialized")
            
        # Extract parameters from config
        config = config or {}
        stop = config.get("stop")
        user_sso = config.get("user_sso")
        
        # Get authentication token if not provided
        if not user_sso:
            try:
                user_sso = gs_auth.get_sso()
                logger.debug("Using automatically retrieved sso token")
            except Exception as e:
                raise ValueError(f"Failed to get sso token: {str(e)}")
            
        # Call the CMS inference API
        answer, _, error = await self.cms.async_inference(
            user_sso=user_sso, 
            question=input
        )

        # Handle errors
        if error is not None:
            raise RuntimeError(f"Error in inference: {error}")

        # Stop token handling is done server-side in the API
            
        return answer
        
    def batch(
        self, 
        inputs: List[str], 
        config: Optional[Dict[str, Any]] = None
    ) -> List[str]:
        """
        Process a batch of inputs.
        
        Args:
            inputs: List of prompts to process
            config: Optional configuration dictionary
            
        Returns:
            List of responses
        """
        return [self.invoke(input, config) for input in inputs]

    async def abatch(
        self, 
        inputs: List[str], 
        config: Optional[Dict[str, Any]] = None
    ) -> List[str]:
        """
        Asynchronously process a batch of inputs.
        
        Args:
            inputs: List of prompts to process
            config: Optional configuration dictionary
            
        Returns:
            List of responses
        """
        return [await self.ainvoke(input, config) for input in inputs]
        
    def stream(self, input: str, config: Optional[Dict[str, Any]] = None) -> Iterator[str]:
        """
        Stream responses from the language model.
        
        Note: Streaming is not currently supported by the CMS API.
        
        Raises:
            NotImplementedError: This functionality is not supported
        """
        raise NotImplementedError(
            "Streaming is not supported for this model"
        )
        
    async def astream(self, input: str, config: Optional[Dict[str, Any]] = None) -> Iterator[str]:
        """
        Asynchronously stream responses from the language model.
        
        Note: Streaming is not currently supported by the CMS API.
        
        Raises:
            NotImplementedError: This functionality is not supported
        """
        raise NotImplementedError(
            "Async streaming is not supported for this model"
        )